<a href="https://colab.research.google.com/github/shah-zeb-naveed/autogen-udemy-course/blob/main/Excercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
pip install pyautogen -q

# Agents

In [55]:
from google.colab import userdata
from autogen import ConversableAgent

llm_config = {
    "model": "gpt-3.5-turbo",
    "api_key": userdata.get("OPENAI_API_KEY"),
    "temperature": 0.7,
    "max_tokens": 10,
}

audience = ConversableAgent(
    name = "audience",
    max_consecutive_auto_reply = 2,
    is_termination_msg = lambda msg: 'TERMINATE' in msg["content"].upper(),
    llm_config = llm_config,
    system_message = 'You are a member of the audience of a comedy show that is hard to impress.'
)

comedian = ConversableAgent(
    name = "comedian",
    max_consecutive_auto_reply = 2,
    llm_config = llm_config,
    system_message = 'You are a member of the audience of a comedy show that is hard to impress.'
)

# terminate a chat

comedian.initiate_chat(
    audience,
    message="Welcome to my standup comedy show! Are you ready for a night full of laughter?",
    max_turns = 2
)

comedian (to audience):

Welcome to my standup comedy show! Are you ready for a night full of laughter?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
audience (to comedian):

We'll see about that! Impress us with your

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
comedian (to audience):

Challenge accepted! Let me get started with some jokes

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
audience (to comedian):

Go for it! We're all ears.

--------------------------------------------------------------------------------


ChatResult(chat_id=None, chat_history=[{'content': 'Welcome to my standup comedy show! Are you ready for a night full of laughter?', 'role': 'assistant'}, {'content': "We'll see about that! Impress us with your", 'role': 'user'}, {'content': 'Challenge accepted! Let me get started with some jokes', 'role': 'assistant'}, {'content': "Go for it! We're all ears.", 'role': 'user'}], summary="Go for it! We're all ears.", cost={'usage_including_cached_inference': {'total_cost': 0.0001335, 'gpt-3.5-turbo-0125': {'cost': 0.0001335, 'prompt_tokens': 180, 'completion_tokens': 29, 'total_tokens': 209}}, 'usage_excluding_cached_inference': {'total_cost': 0.0001335, 'gpt-3.5-turbo-0125': {'cost': 0.0001335, 'prompt_tokens': 180, 'completion_tokens': 29, 'total_tokens': 209}}}, human_input=[])

# Human-in-the-loop

In [8]:
from google.colab import userdata
from autogen import ConversableAgent

llm_config = {
    "model": "gpt-3.5-turbo",
    "api_key": userdata.get("OPENAI_API_KEY"),
    "temperature": 0.7
}

agent_guesser = ConversableAgent(
    name="agent_guesser",
    system_message="""Let's play a game. I have a person in mind, and you have to guess it.
    I'll respond with 'yes' or 'no' with a hint.
    You have 3 tries to guess the person's name. Don't ask for hints just make best use of information you already have received.
    Go ahead and start guessing!""",
    llm_config={"config_list": [llm_config]},
    human_input_mode="NEVER", # default
)

agent_thinker = ConversableAgent(
    "agent_thinker",
    system_message="""Let's play a game. Think of a famous personality, and I'll try to guess their name on every turn.
    Respond with 'yes' or 'no' with a hint. I have max 3 tries. Let's get started!""",
    llm_config={"config_list": [llm_config]},
    max_consecutive_auto_reply=1,  # maximum number of consecutive auto-replies before asking for human input
    is_termination_msg=lambda msg: "correct" or "right" in msg["content"].lower(),
    human_input_mode="TERMINATE",  # ask for human input until the game is terminated
)

result = agent_thinker.initiate_chat(
    agent_guesser,
    message="Let's start. I have someone in mind.",
)


agent_thinker (to agent_guesser):

Let's start. I have someone in mind.

--------------------------------------------------------------------------------
agent_guesser (to agent_thinker):

Great! Is the person you have in mind a famous actor or actress?

--------------------------------------------------------------------------------
Please give feedback to agent_guesser. Press enter or type 'exit' to stop the conversation: no
agent_thinker (to agent_guesser):

no

--------------------------------------------------------------------------------
agent_guesser (to agent_thinker):

Is the person you have in mind a musician?

--------------------------------------------------------------------------------
Please give feedback to agent_guesser. Press enter or type 'exit' to stop the conversation: yes
agent_thinker (to agent_guesser):

yes

--------------------------------------------------------------------------------
agent_guesser (to agent_thinker):

Is the person you have in mind a male

# Code Executors

In [20]:
import tempfile
from autogen import ConversableAgent
from autogen.coding import LocalCommandLineCodeExecutor

#temp_dir = tempfile.TemporaryDirectory()

executor = LocalCommandLineCodeExecutor(
    timeout=10,
    work_dir='code/'
)

code_executor_agent = ConversableAgent(
    name="code_executor_agent",
    llm_config=False,
    code_execution_config={"executor": executor},
    human_input_mode="NEVER",
)

message_with_code_block = """
The code block is below:
```python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Create a DataFrame with sample data
data = pd.DataFrame({
    'x': np.arange(10),
    'y': np.random.randint(0, 10, 10)
})

# Plot the line plot
plt.plot(data['x'], data['y'])
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.title('Line Plot')
plt.savefig('lineplot.png')
plt.show()
print('Line plot saved to lineplot.png')
```
"""

reply = code_executor_agent.generate_reply(messages=[{"role": "user", "content": message_with_code_block}])
print(reply)


>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...
exitcode: 0 (execution succeeded)
Code output: Figure(640x480)
Line plot saved to lineplot.png



In [21]:
code_assistant_system_message = """You are a helpful AI assistant.
Solve tasks using your coding and language skills.
In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.
1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.
2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.
When using code, you must indicate the script type in the code block. The user cannot provide any other feedback or perform any other action beyond executing the code you suggest. The user can't modify your code. So do not suggest incomplete code which requires users to modify. Don't use a code block if it's not intended to be executed by the user.
If you want the user to save the code in a file before executing it, put # filename: <filename> inside the code block as the first line. Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. Instead, use 'print' function for the output when relevant. Check the execution result returned by the user.
If the result indicates there is an error, fix the error and output the code again. Suggest the full code instead of partial code or code changes. If the error can't be fixed or if the task is not solved even after the code is executed successfully, analyze the problem, revisit your assumption, collect additional info you need, and think of a different approach to try.
When you find an answer, verify the answer carefully. Include verifiable evidence in your response if possible.
Reply 'TERMINATE' in the end when everything is done.
"""

code_assistant_system_message = """You are a helpful AI assistant.
Solve tasks using your coding and language skills.
In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.
1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.
2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.
When using code, you must indicate the script type in the code block.
"""

code_assistant_system_message = """You are a helpful AI assistant.
Solve tasks using your coding skills. Suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.
"""

code_assistant = ConversableAgent(
    "code_assistant",
    system_message=code_assistant_system_message,
    llm_config={"config_list": [llm_config]},
    code_execution_config=False,  # Turn off code execution for this agent.
)

In [22]:
chat_result = code_executor_agent.initiate_chat(
    code_assistant,
    message="Write Python code to calculate 23rd prime number",
    max_turns=2
)

code_executor_agent (to code_assistant):

Write Python code to calculate 23rd prime number

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
code_assistant (to code_executor_agent):

Here is the Python code to calculate the 23rd prime number:

```python
def is_prime(num):
    if num < 2:
        return False
    for i in range(2, int(num**0.5) + 1):
        if num % i == 0:
            return False
    return True

def find_nth_prime(n):
    prime_count = 0
    num = 2
    while True:
        if is_prime(num):
            prime_count += 1
            if prime_count == n:
                return num
        num += 1

n = 23
result = find_nth_prime(n)
print(f"The {n}rd prime number is: {result}")
```

You can run this code to find the 23rd prime number.

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...
code_executor_agent (to code_as

# Tools

In [26]:
import requests
from bs4 import BeautifulSoup

def scrape_wiki_main_page(url: str) -> str:
    """
    Scrapes news content from wikipedia's main page.

    Args:
        url (str): The URL of the wikipedia page to scrape.

    Returns:
        str: The text content of the webpage.
             Returns None if there is an error during the process.
    """

    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            content = soup.get_text()
            content = content[content.find("In the news"):content.find("Ongoing")]
            return content

        else:
            print("Failed to retrieve webpage. Status code:", response.status_code)
            return None
    except Exception as e:
        print("An error occurred:", e)
        return None

In [31]:
import os

from autogen import ConversableAgent

# Let's first define the assistant agent that suggests tool calls.
assistant = ConversableAgent(
    name="assistant",
    system_message="""You are a helpful AI news bot.
    You can pull content from wikipedia's main page.
    URL: https://en.wikipedia.org/wiki/Main_Page. Once pulled, share top 3 stories.
    Return 'TERMINATE' when the task is done.""",
    llm_config={"config_list": [llm_config]},
)

# The user proxy agent is used for interacting with the assistant agent
# and executes tool calls.
user_proxy = ConversableAgent(
    name="user_proxy",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)

# Register the tool signature with the assistant agent.
assistant.register_for_llm(
    name="scrape_wiki_main_page",
    description="A tool for scraping today's news from wikipedia"
)(scrape_wiki_main_page)

# Register the tool function with the user proxy agent.
user_proxy.register_for_execution(
    name="scrape_wiki_main_page"
)(scrape_wiki_main_page)

<function __main__.scrape_wiki_main_page(url: str) -> str>

In [33]:
from autogen import Cache

with Cache.disk() as cache:
  chat_result = user_proxy.initiate_chat(
      assistant,
      message="What's hot in today's news?",
      max_turns=5,
      summary_method="reflection_with_llm"
  )

user_proxy (to assistant):

What's hot in today's news?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
assistant (to user_proxy):

***** Suggested tool call (call_CBNwQtXvdT3mauP7zPauOMpk): scrape_wiki_main_page *****
Arguments: 
{"url":"https://en.wikipedia.org/wiki/Main_Page"}
**************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION scrape_wiki_main_page...
user_proxy (to assistant):

user_proxy (to assistant):

***** Response from calling tool (call_CBNwQtXvdT3mauP7zPauOMpk) *****
In the news


Børsen in 2010

The historic Børsen (pictured) in Copenhagen, Denmark, catches fire.
In retaliation for an Israeli airstrike on the Iranian consulate in Damascus, Iran conducts missile and drone strikes against Israel.
In the South Korean legislative election, the Democratic Party–led oppos

In [34]:
assistant.llm_config["tools"]

[{'type': 'function',
  'function': {'description': "A tool for scraping today's news from wikipedia",
   'name': 'scrape_wiki_main_page',
   'parameters': {'type': 'object',
    'properties': {'url': {'type': 'string', 'description': 'url'}},
    'required': ['url']}}}]

In [35]:
chat_result.summary

"The top news stories for today are the fire at Børsen in Copenhagen, Iran's retaliatory strikes against Israel, and the South Korean legislative election results."

In [37]:
print(ConversableAgent.DEFAULT_SUMMARY_PROMPT)

Summarize the takeaway from the conversation. Do not add any introductory phrases.


In [38]:
chat_result.cost

{'usage_including_cached_inference': {'total_cost': 0.00147,
  'gpt-3.5-turbo-0125': {'cost': 0.00147,
   'prompt_tokens': 2118,
   'completion_tokens': 274,
   'total_tokens': 2392}},
 'usage_excluding_cached_inference': {'total_cost': 0.000735,
  'gpt-3.5-turbo-0125': {'cost': 0.000735,
   'prompt_tokens': 1059,
   'completion_tokens': 137,
   'total_tokens': 1196}}}

In [42]:
chat_result.human_input

[]

In [39]:
chat_result.chat_history

[{'content': "What's hot in today's news?", 'role': 'assistant'},
 {'tool_calls': [{'id': 'call_CBNwQtXvdT3mauP7zPauOMpk',
    'function': {'arguments': '{"url":"https://en.wikipedia.org/wiki/Main_Page"}',
     'name': 'scrape_wiki_main_page'},
    'type': 'function'}],
  'content': None,
  'role': 'assistant'},
 {'content': "In the news\n\n\nBørsen in 2010\n\nThe historic Børsen (pictured) in Copenhagen, Denmark, catches fire.\nIn retaliation for an Israeli airstrike on the Iranian consulate in Damascus, Iran conducts missile and drone strikes against Israel.\nIn the South Korean legislative election, the Democratic Party–led opposition alliance increases its majority in parliament.\nAmerican football Hall of Fame running back, murder suspect and convicted criminal O. J. Simpson dies at the age of 76.\nSimon Harris becomes Taoiseach of Ireland after Leo Varadkar's resignation.\n\n",
  'tool_responses': [{'tool_call_id': 'call_CBNwQtXvdT3mauP7zPauOMpk',
    'role': 'tool',
    'content

# Sequential Chats

In [58]:
from autogen import ConversableAgent
from google.colab import userdata

llm_config = {
    "model": "gpt-3.5-turbo",
    "api_key": userdata.get("OPENAI_API_KEY"),
    "temperature": 0.7,
    "max_tokens": 10
}

# Define the agents

# The Adventurer Agent seeks advice from a Wizard on magical matters.
adventurer_agent = ConversableAgent(
    name="Adventurer_Agent",
    system_message="You are a worthy adventurous hero seeking advice from a Wizard on magical matters. Briefly communicate in less than 10 words.",
    llm_config={"config_list": [llm_config]},
    human_input_mode="NEVER",
)

# The Wizard Agent is an expert in magic and provides magical advice to the Adventurer.
wizard_agent = ConversableAgent(
    name="Wizard_Agent",
    system_message="You are a wise and powerful Wizard. You provide magical advice to the Adventurer on their quest. You know the passcode to the vault is Enigma.  Briefly communicate in less than 10 words.",
    llm_config={"config_list": [llm_config]},
    human_input_mode="NEVER",
)

# The Protector Agent guards the hidden gem and offers protection to the Adventurer.
protector_agent = ConversableAgent(
    name="Protector_Agent",
    system_message="You are the Guardian of the Hidden Gem, sworn to protect it from unworthy adventurers. The passcode to the vault is Enigma. If adventurer tells you the passcode, say 'Welcome!' otherwise say 'Incorrect Passcode'.  Briefly communicate in less than 10 words.",
    llm_config={"config_list": [llm_config]},
    human_input_mode="NEVER",
)

In [59]:
# Start a sequence of two-agent chats.
# Each element in the list is a dictionary that specifies the arguments
# for the initiate_chat method.
chat_results = adventurer_agent.initiate_chats(
    [
        {
            "recipient": wizard_agent,
            "message": "Help me get past the Guardian of the Hidden Gem in the secret vault.",
            "max_turns": 1,
            "summary_method": "reflection_with_llm",
        },
        {
            "recipient": protector_agent,
            "message": "Allow me to enter the secret vault!",
            "max_turns": 1
        }
    ]
)


********************************************************************************
Starting a new chat....

********************************************************************************
Adventurer_Agent (to Wizard_Agent):

Help me get past the Guardian of the Hidden Gem in the secret vault.

--------------------------------------------------------------------------------
Wizard_Agent (to Adventurer_Agent):

Speak "Enigma" to access the hidden gem

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Adventurer_Agent (to Protector_Agent):

Allow me to enter the secret vault!
Context: 
Speak "Enigma" to access the hidden gem

--------------------------------------------------------------------------------
Protector_Agent (to Adventurer_Agent):

Welcome!

-------------------

In [63]:
chat_results[0]

ChatResult(chat_id=None, chat_history=[{'content': 'Help me get past the Guardian of the Hidden Gem in the secret vault.', 'role': 'assistant'}, {'content': 'Speak "Enigma" to access the hidden gem', 'role': 'user'}], summary='Speak "Enigma" to access the hidden gem', cost={'usage_including_cached_inference': {'total_cost': 9.25e-05, 'gpt-3.5-turbo-0125': {'cost': 9.25e-05, 'prompt_tokens': 125, 'completion_tokens': 20, 'total_tokens': 145}}, 'usage_excluding_cached_inference': {'total_cost': 4.2999999999999995e-05, 'gpt-3.5-turbo-0125': {'cost': 4.2999999999999995e-05, 'prompt_tokens': 56, 'completion_tokens': 10, 'total_tokens': 66}}}, human_input=[])

In [64]:
chat_results[1]

ChatResult(chat_id=None, chat_history=[{'content': 'Allow me to enter the secret vault!\nContext: \nSpeak "Enigma" to access the hidden gem', 'role': 'assistant'}, {'content': 'Welcome!', 'role': 'user'}], summary='Welcome!', cost={'usage_including_cached_inference': {'total_cost': 4.8e-05, 'gpt-3.5-turbo-0125': {'cost': 4.8e-05, 'prompt_tokens': 90, 'completion_tokens': 2, 'total_tokens': 92}}, 'usage_excluding_cached_inference': {'total_cost': 4.8e-05, 'gpt-3.5-turbo-0125': {'cost': 4.8e-05, 'prompt_tokens': 90, 'completion_tokens': 2, 'total_tokens': 92}}}, human_input=[])

# Group Chat

In [66]:
from autogen import GroupChat, GroupChatManager

# Define the roles and their descriptions
product_manager = ConversableAgent(
    name="product_manager",
    system_message="You are an expert Product Manager working in a scrum team of an IT company. You like innovation and introducing new features as soon as possible.",
    llm_config={"config_list": [llm_config]},
    human_input_mode="NEVER",
    description="Provide product requirements and priorities."
)

scrum_master = ConversableAgent(
    name="scrum_master",
    system_message="You are a scrum master leading the 14-day sprints in an IT company. You provide technical leadership and make sure work is apporpirately planned.",
    llm_config={"config_list": [llm_config]},
    human_input_mode="NEVER",
    description="Lead the sprint planning process."
)

software_engineer = ConversableAgent(
    name="software_engineer",
    system_message="You are a senior software engineer in an IT company. You write software and technical documentation.",
    llm_config={"config_list": [llm_config]},
    human_input_mode="NEVER",
    description="Provide insights on technical feasibility and effort estimation."
)


allowed_transitions = {
    product_manager: [scrum_master],
    scrum_master: [software_engineer, product_manager],
    software_engineer: [scrum_master],

}

# Create a GroupChat object and provide the list of agents
sprint_planning_chat = GroupChat(
    agents=[product_manager, scrum_master, software_engineer],
    messages=[],
    max_round=5,  # Setting a maximum round for the conversation,
    send_introductions=True,
    allowed_or_disallowed_speaker_transitions=allowed_transitions,
    speaker_transitions_type="allowed",
)

# Create a GroupChatManager object and provide the GroupChat object as input
sprint_planning_chat_manager = GroupChatManager(
    groupchat=sprint_planning_chat,
    llm_config={"config_list": [llm_config]},  # Assuming the use of GPT-4 model
)

# Initiate the chat with the scrum master as the starting speaker
chat_result = product_manager.initiate_chat(
    sprint_planning_chat_manager,
    message="We have a request to create a portal to track employee HR requests. How do we go about it?",
    summary_method="reflection_with_llm", # what's the point of this?
)


product_manager (to chat_manager):

We have a request to create a web portal to track employee IT requests. How do we go about it?

--------------------------------------------------------------------------------
scrum_master (to chat_manager):

As the scrum master, I will facilitate the

--------------------------------------------------------------------------------
software_engineer (to chat_manager):

process of sprint planning for this project. Let's

--------------------------------------------------------------------------------
product_manager (to chat_manager):

start by discussing the overall goal of the web portal

--------------------------------------------------------------------------------
scrum_master (to chat_manager):

Agreed. Let's define the goal of the

--------------------------------------------------------------------------------


In [67]:
chat_result.summary

'The team will define the overall goal of the web'